
# Loading and normalizing CIFAR10


In [ ]:
!pip install albumentations==0.5.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 KB 5.1 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1


In [ ]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
from functools import partial
%matplotlib inline

cv = partial(nn.Conv2d, bias=False)
bn = nn.BatchNorm2d
relu = nn.ReLU

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensor

In [ ]:
transform = transforms.Compose([transforms.ToTensor()]) # simple ToTensor for now

train_set = torchvision.datasets.CIFAR10(root='./cifar10', download=True, train=True, transform=transform)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10


In [ ]:
train_set.data.shape # train data shape

(50000, 32, 32, 3)

In [ ]:
# calculate mean and standard deviation

m = np.mean(train_set.data, axis=(0,1,2))
print(m/255)

s = np.std(train_set.data, axis=(0, 1, 2))
print(s/255)

[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [ ]:
mean = m/255
sdev = s/255

In [ ]:
SEED = 101
BATCH = 32

cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

if cuda:
    torch.cuda.manual_seed(SEED)

device = torch.device("cuda" if cuda else "cpu")

# dataloader arguments
dataloader_args = dict(shuffle=True, batch_size=BATCH, num_workers=2, pin_memory=True)\
    if cuda else dict(shuffle=True, batch_size=32)


CUDA Available? False


In [ ]:
def train_eval_model(model, train_loader, optimizer, device, epochs=1, test=False, test_loader=None, scheduler=None):
          
    model.train() # set the train mode
    
    # iterate over for `epochs` epochs and keep storing valuable info

    for epoch in range(epochs):
      correct = processed = train_loss = 0
    #   if scheduler:
    #     scheduler.step()

      print(f"\n epoch num ================================= {epoch+1}")
      
      pbar = tqdm(train_loader)
      
      for batch_idx, (data, target) in enumerate(pbar):
          data, target = data.to(device), target.to(device) # move data to `device`

          optimizer.zero_grad() # zero out the gradients to avoid accumulating them over loops

          output = model(data) # get the model's predictions

          loss = F.nll_loss(output, target) # calculate Negative Log Likelihood loss using ground truth labels and the model's predictions

          train_loss += loss.item() # add up the train loss

          loss.backward() # The magic function to perform backpropagation and calculate the gradients

          optimizer.step() # take 1 step for the optimizer and update the weights

          pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability

          correct += pred.eq(target.view_as(pred)).sum().item() #  compare and see how many predictions are coorect and then add up the count

          processed += len(data) # total processed data size

      acc = 100 * correct/processed

      train_losses.append(train_loss)

      train_accuracy.append(acc)

      if scheduler:
        print("\n\n\t\t\tLast LR -->", scheduler.get_last_lr())
        scheduler.step()

      pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
      
      
      train_loss /= len(train_loader.dataset)
      
      print('\n\t\t\tTrain metrics: accuracy: {}/{} ({:.4f}%)'.format(correct,
                                                              len(train_loader.dataset),
                                                              correct * 100 / len(train_loader.dataset)))

      if test: # moving to evaluation
          model.eval() # set the correct mode
          
          correct = test_loss = 0

          with torch.no_grad(): # to disable gradient calculation with no_grad context
              
              for data, target in test_loader:

                  data, target = data.to(device), target.to(device)
                  
                  output = model(data)
                  
                  test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
                  
                  pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                  
                  correct += pred.eq(target.view_as(pred)).sum().item()

          test_loss /= len(test_loader.dataset)
          test_losses.append(test_loss)
          test_accuracy.append(100. * correct / len(test_loader.dataset))

          print('\n\tTest metrics: average loss: {:.4f}, accuracy: {}/{} ({:.5f}%)\n'.format(
              test_loss, correct, len(test_loader.dataset),
              100. * correct / len(test_loader.dataset)))

In [ ]:
def plot_graphs(*, train_losses, train_accuracy, test_losses, test_accuracy):
    fig, axs = plt.subplots(2,2,figsize=(20,10))
    axs[0, 0].plot(train_losses)
    axs[0, 0].set_title("Training Loss")
    axs[1, 0].plot(train_accuracy)
    axs[1, 0].set_title("Training Accuracy")
    axs[0, 1].plot(test_losses)
    axs[0, 1].set_title("Test Loss")
    axs[1, 1].plot(test_accuracy)
    axs[1, 1].set_title("Test Accuracy")

In [ ]:
class Cifar10SearchDataset(torchvision.datasets.CIFAR10):
    def __init__(self, root="./data/cifar10", train=True, download=True, transform=None):
        super().__init__(root=root, train=train, download=download, transform=transform)

    def __getitem__(self, index):
        image, label = self.data[index], self.targets[index]

        if self.transform is not None:
            transformed = self.transform(image=image)
            image = transformed["image"]

        return image, label

In [ ]:
# transforms

# here changed CoarseDropout's p to 0.25 from default 0.5
train_transforms = A.Compose([
                                A.HorizontalFlip(p=0.2),
                                A.ShiftScaleRotate(shift_limit=0.1,scale_limit=0.15,rotate_limit=30,p=0.20),
                                A.CoarseDropout(max_holes=1, p=0.15, max_height=16,
                                                max_width=16, min_holes=1, min_height=16,
                                                min_width=16, fill_value=mean),
                                # A.MedianBlur(blur_limit=3, p=0.1),
                                A.HueSaturationValue(p=0.1),
                            #   A.GaussianBlur(blur_limit=3, p=0.12),
                                # A.RandomBrightnessContrast(brightness_limit=0.09,contrast_limit=0.1, p=0.15),   
                                A.Normalize(mean=mean, std=sdev),
                                ToTensor()
                                       ])

test_transforms = A.Compose([
                            A.Normalize(mean=mean, std=sdev),
                            ToTensor()
                                    ])


In [ ]:

# data sets with transforms
train_set = Cifar10SearchDataset(train=True, download=True, transform=train_transforms)

test_set = Cifar10SearchDataset(train=False, download=True, transform=test_transforms)


# data loaders on data sets
train_loader = torch.utils.data.DataLoader(dataset=train_set, **dataloader_args)

test_loader = torch.utils.data.DataLoader(test_set, **dataloader_args)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar10/cifar-10-python.tar.gz to ./data/cifar10
Files already downloaded and verified


In [ ]:
from __future__ import print_function
# channel_means = (0.49196659, 0.48229005, 0.4461573)
# channel_stdevs = (0.24703223, 0.24348513, 0.26158784)
def unnormalize(img):
  img = img.numpy().astype(dtype=np.float32)
  
  for i in range(img.shape[0]):
    img[i] = (img[i]*sdev[i])+mean[i]
  
  return np.transpose(img, (1,2,0))

%matplotlib inline
import matplotlib.pyplot as plt
dataiter = iter(train_loader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

num_classes = 10
# display 10 images from each category. 
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
r, c = 10, 11
n = 5
fig = plt.figure(figsize=(15,15))
fig.subplots_adjust(hspace=0.2, wspace=0.2)
for i in range(num_classes):
    idx = np.random.choice(np.where(labels[:]==i)[0], n)
    ax = plt.subplot(r, c, i*c+1)
    ax.text(-1.5, 0.5, class_names[i], fontsize=14)
    plt.axis('off')
    for j in range(1, n+1):
      plt.subplot(r, c, i*c+j+1)
      plt.imshow(unnormalize(images[idx[j-1]]), interpolation='none')
      plt.axis('off')
plt.show()

AttributeError: ignored

# Code 10

In [ ]:
class Net10(nn.Module):
    def __init__(self):
        super().__init__()

        self.block1 = nn.Sequential(
            cv(3, 16, 3, padding=1),
            relu(),
            bn(16),
            cv(16, 16, 1),
            relu(),
            bn(16),
            cv(16, 32, 3, dilation=1, groups=1, padding=1),
            relu(),
            bn(32),
            cv(32, 32, 1),
            relu(),
            bn(32),
            nn.Dropout2d(0.05)
        )

        self.block2 = nn.Sequential(
            cv(32, 64, 3, padding=1, dilation=1, groups=32),
            relu(),
            bn(64),
            cv(64, 32, 1),
            relu(),
            bn(32),
            cv(32, 64, 3, dilation=1, padding=1, groups=16),
            relu(),
            bn(64),
            cv(64, 48, 1),
            relu(),
            bn(48),
            cv(48, 64, 3, padding=1, dilation=2, groups=8),
            relu(),
            bn(64),
            cv(64, 32, 1),
            relu(),
            bn(32),
            nn.Dropout2d(0.08)
        )

        self.block3 = nn.Sequential(
            cv(32, 64, 3, dilation=1, padding=1, groups=32, stride=1),
            relu(),
            bn(64),
            cv(64, 128, 3, groups=16, dilation=1, padding=1),
            relu(),
            bn(128),
            cv(128, 64, 1),
            relu(),
            bn(64),
            cv(64, 96, 3, padding=1, groups=32),
            relu(),
            bn(96),
            cv(96, 64, 1),
            relu(),
            bn(64),
            cv(64, 64, 3, padding=2, dilation=2, stride=2),
            relu(),
            bn(64),
            cv(64, 64, 1),
            relu(),
            bn(64),
            nn.Dropout2d(0.09)
            )

        self.block4 = nn.Sequential(
            cv(64, 96, 3, padding=0, groups=32, stride=1, dilation=2),
            bn(96),
            relu(),
            cv(96, 64, 1),
            bn(64),
            relu(),
            nn.Dropout2d(0.05),
            cv(64, 64, 3, groups=64, padding=0, dilation=2), # depthwise (a)
            cv(64, 32, 1), # pointwise for preceding depthwise (b)
            bn(32),
            relu(),
            cv(32, 48, 3, dilation=2, groups=8),
            relu(),
            bn(48),
            cv(48, 10, 1, stride=1),
            relu(),
            bn(10),
            nn.AdaptiveAvgPool2d(1)
            # cv(10, 10, 1)
        )
    
        
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)


model = Net10().to(device)
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
            Conv2d-4           [-1, 16, 32, 32]             256
              ReLU-5           [-1, 16, 32, 32]               0
       BatchNorm2d-6           [-1, 16, 32, 32]              32
            Conv2d-7           [-1, 32, 32, 32]           4,608
              ReLU-8           [-1, 32, 32, 32]               0
       BatchNorm2d-9           [-1, 32, 32, 32]              64
           Conv2d-10           [-1, 32, 32, 32]           1,024
             ReLU-11           [-1, 32, 32, 32]               0
      BatchNorm2d-12           [-1, 32, 32, 32]              64
        Dropout2d-13           [-1, 32, 32, 32]               0
           Conv2d-14           [-1, 64,

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, nesterov=True)
# step_lr = StepLR(optimizer=optimizer, step_size=1, gamma=0.87)

train_losses = []
train_accuracy = []
test_losses = []
test_accuracy = []

train_eval_model(model, train_loader, optimizer, device, epochs=50, test=True, test_loader=test_loader,scheduler=None)
torch.save(model, "./eva8_s6_exp_2.pth")


 epoch num ================================= 1


100%|██████████| 1563/1563 [01:05<00:00, 23.73it/s]


			Train metrics: accuracy: 6949/50000 (13.8980%)



	Test metrics: average loss: 2.1395, accuracy: 2017/10000 (20.17000%)


 epoch num ================================= 2


100%|██████████| 1563/1563 [01:06<00:00, 23.42it/s]


...


			Train metrics: accuracy: 35107/50000 (70.2140%)



	Test metrics: average loss: 0.8014, accuracy: 7209/10000 (72.09000%)


 epoch num ================================= 50


100%|██████████| 1563/1563 [01:03<00:00, 24.57it/s]


			Train metrics: accuracy: 35275/50000 (70.5500%)



	Test metrics: average loss: 0.8059, accuracy: 7192/10000 (71.92000%)



In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)

train_eval_model(model, train_loader, optimizer, device, epochs=10, test=True, test_loader=test_loader,scheduler=None)
torch.save(model, "./eva8_s6_exp_2.pth")


 epoch num ================================= 1


100%|██████████| 1563/1563 [01:05<00:00, 24.02it/s]


			Train metrics: accuracy: 20572/50000 (41.1440%)



	Test metrics: average loss: 1.2302, accuracy: 5611/10000 (56.11000%)


 epoch num ================================= 2


100%|██████████| 1563/1563 [01:03<00:00, 24.80it/s]

...


			Train metrics: accuracy: 35567/50000 (71.1340%)



	Test metrics: average loss: 0.7374, accuracy: 7469/10000 (74.69000%)


 epoch num ================================= 10


100%|██████████| 1563/1563 [01:02<00:00, 24.93it/s]


			Train metrics: accuracy: 36041/50000 (72.0820%)



	Test metrics: average loss: 0.6951, accuracy: 7576/10000 (75.76000%)



In [ ]:
from google.colab import files

files.download( "./eva8_s6_exp_2.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)

train_eval_model(model, train_loader, optimizer, device, epochs=20, test=True, test_loader=test_loader,scheduler=None)
torch.save(model, "./eva8_s6_exp_2.pth")


 epoch num ================================= 1


100%|██████████| 1563/1563 [01:08<00:00, 22.91it/s]


			Train metrics: accuracy: 33432/50000 (66.8640%)



	Test metrics: average loss: 0.6537, accuracy: 7787/10000 (77.87000%)


 epoch num ================================= 2


100%|██████████| 1563/1563 [01:04<00:00, 24.06it/s]

...


			Train metrics: accuracy: 39867/50000 (79.7340%)



	Test metrics: average loss: 0.5872, accuracy: 8043/10000 (80.43000%)


 epoch num ================================= 20


100%|██████████| 1563/1563 [01:02<00:00, 24.90it/s]


			Train metrics: accuracy: 39820/50000 (79.6400%)



	Test metrics: average loss: 0.5456, accuracy: 8125/10000 (81.25000%)



In [ ]:
files.download( "./eva8_s6_exp_2.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.0003, momentum=0.9, nesterov=True)

train_eval_model(model, train_loader, optimizer, device, epochs=50, test=True, test_loader=test_loader,scheduler=None)

torch.save(model, "./eva8_s6_exp_2_lr0003_epochs40.pth")
files.download("./eva8_s6_exp_2_lr0003_epochs40.pth")


 epoch num ================================= 1


100%|██████████| 1563/1563 [01:04<00:00, 24.13it/s]


			Train metrics: accuracy: 33009/50000 (66.0180%)



	Test metrics: average loss: 0.6735, accuracy: 7995/10000 (79.95000%)


 epoch num ================================= 2


100%|██████████| 1563/1563 [01:02<00:00, 24.86it/s]

...

100%|██████████| 1563/1563 [01:02<00:00, 24.90it/s]


			Train metrics: accuracy: 43505/50000 (87.0100%)



	Test metrics: average loss: 0.4630, accuracy: 8457/10000 (84.57000%)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)

step_lr = StepLR(optimizer=optimizer, step_size=2, gamma=0.81, verbose=True)

train_eval_model(model, train_loader, optimizer, device, epochs=50, test=True,
                 test_loader=test_loader,
                 scheduler=step_lr)

torch.save(model, "./eva8_s6_steplr1_epochs50.pth")
files.download("./eva8_s6_steplr1_epochs50.pth")

Adjusting learning rate of group 0 to 1.0000e-03.

 epoch num ================================= 1


100%|██████████| 1563/1563 [01:04<00:00, 24.18it/s]



			Last LR --> [0.001]
Adjusting learning rate of group 0 to 1.0000e-03.

			Train metrics: accuracy: 35712/50000 (71.4240%)



	Test metrics: average loss: 0.5265, accuracy: 8308/10000 (83.08000%)


 epoch num ================================= 2


100%|██████████| 1563/1563 [01:02<00:00, 25.11it/s]

...

100%|██████████| 1563/1563 [01:02<00:00, 25.13it/s]



			Last LR --> [0.00012157665459056933]
Adjusting learning rate of group 0 to 9.8477e-05.

			Train metrics: accuracy: 43823/50000 (87.6460%)



	Test metrics: average loss: 0.4580, accuracy: 8492/10000 (84.92000%)


 epoch num ================================= 23


  5%|▍         | 77/1563 [00:03<01:01, 23.97it/s]


KeyboardInterrupt: ignored

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9, nesterov=True)

step_lr = StepLR(optimizer=optimizer, step_size=1, gamma=0.81)

train_eval_model(model, train_loader, optimizer, device, epochs=20, test=True, test_loader=test_loader,scheduler=step_lr)

torch.save(model, "./eva8_s6_steplr2_epochs20.pth")
files.download("./eva8_s6_steplr2_epochs20.pth")


 epoch num ================================= 1


100%|██████████| 1563/1563 [01:06<00:00, 23.48it/s]



			Last LR --> [0.003]

			Train metrics: accuracy: 36784/50000 (73.5680%)



	Test metrics: average loss: 0.4809, accuracy: 8382/10000 (83.82000%)


 epoch num ================================= 2


100%|██████████| 1563/1563 [01:03<00:00, 24.60it/s]

...

100%|██████████| 1563/1563 [01:02<00:00, 25.04it/s]



			Last LR --> [5.4744010894202264e-05]

			Train metrics: accuracy: 44202/50000 (88.4040%)



	Test metrics: average loss: 0.4577, accuracy: 8504/10000 (85.04000%)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
# plot_graphs(train_losses=train_losses, train_accuracy=train_accuracy,
#             test_losses=test_losses,test_accuracy=test_accuracy)